In [1]:
import os
os.chdir('../../src/models')
import simulate_pandemic as simp
from policies import policies, costs
from tqdm import tqdm
import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
import datetime

from policies import costs

set3 = px.colors.qualitative.Set3

color_map = {
    'Lockdown':          set3[3],
    'Hard Quarantine':    set3[11],
    'Light Quarantine':   set3[1],
    'Social Distancing':  'rgb(204, 245, 175)',
    'Unrestricted':        set3[6]     
}

Loading Graph... Done!


In [20]:
def make_SIR_graph(data):
    d_counts = pd.DataFrame([pd.Series(d).value_counts() for d in data])
    d_counts.fillna(0, inplace=True)

    fig = go.Figure()
    x = pd.date_range(datetime.date(2020, 2, 24), periods=len(d_counts[0]))

    fig.add_trace(go.Scatter(x=x, y=d_counts[-1]/55e3, name='removed',line_color='green'))
    fig.add_trace(go.Scatter(x=x, y=d_counts[0]/55e3, name='susceptible', line_color='blue'))
    fig.add_trace(go.Scatter(x=x, y=d_counts[1]/55e3, name='exposed',line_color ='orange'))
    fig.add_trace(go.Scatter(x=x, y=d_counts[2]/55e3, name='infected', line_color = 'red'))
    fig.add_trace(go.Scatter(x=x, y=d_counts[3]/55e3, name='hospitalized', line_color = 'purple'))
    fig.update_layout(hovermode='x')
    fig.show()
    
    return d_counts

def make_beds_graph(data, actions, step_size, title, color_map):
    fig = go.Figure()
    
    x = pd.date_range(datetime.date(2020, 2, 24), periods=len(data)+2)

    fig.add_trace(go.Scatter(x=x, y=data[3]/55e3, name='hospitalized', line_color = 'royalblue',
                            line=dict(width=3)))
    #fig.add_trace(go.Scatter(x=x, y=data[1]/55e3, name='exposed', line_color = 'firebrick',
    #                        line=dict(width=3)))
    fig.add_trace(go.Scatter(x=x, y=len(data)*[0.0025], name='capacity', line_color = 'black',
                            line=dict(dash='dash', width = 2)))
    fig.update_layout(
        shapes=[
            dict(
                type="rect",
                # x-reference is assigned to the x-values
                xref="x",
                # y-reference is assigned to the plot paper [0,1]
                yref="paper",
                x0=x[step_size*i],
                y0=0,
                x1=x[step_size*(i+1)-1],
                y1=1,
                fillcolor=a,
                opacity=0.5,
                layer="below",
                line_width=0,
            ) for i,a in enumerate(actions)] 
    )

    for k,v in color_map.items():
        fig.add_trace(go.Bar(x=[None], y=[None], marker=dict(color=v), name = k))

    fig.update_layout(coloraxis = {'colorscale':'deep'}, xaxis={'showgrid': False},
                      yaxis = {'showgrid': False},
                      showlegend=True, title = title, hovermode="x")
    
    fig.update_layout( 
        title=f'Hospitalized with {title}',
        font=dict(
            size=18,
        )
    )
    
    fig.write_image(f"{title}.png",scale=3)


    fig.show()

In [21]:
def run_simulation(policy, step_size = 7, days = 280):

    pop_matrix = simp.init_infection(.0001)
    data = []
    actions = []
    
    for day in tqdm(range(1, days+1)):
        #if less than 20% still susceptible, break simulation
        if pop_matrix[np.where(pop_matrix[:,1] == -1)].shape[0] > pop_matrix.shape[0]*.9: break
        
        
        # Choose a new policy at each week
        if day % step_size == 1:
            restrictions = policies[policy(day)]
            actions.append(policy(day))
        
        pop_matrix = simp.spread_infection(pop_matrix, restrictions, day)
        pop_matrix = simp.lambda_leak_expose(pop_matrix, day)
        pop_matrix = simp.update_population(pop_matrix)

        data.append(np.array(sorted(pop_matrix,key=lambda x: x[0]))[:,1]) 
    
    return data, actions

In [22]:
def unrestricted(day):
    return 'Unrestricted'

def unrestricted_hard_unrestricted(day):
    if day > 29 and day <= 141: 
        return 'Hard Quarantine'
    return 'Social Distancing'

def intermitent_rest(day):
    if (day-1)%2 == 0:
        return 'Hard Quarantine'
    return 'Social Distancing'

In [24]:
for p,title in zip([unrestricted, unrestricted_hard_unrestricted, intermitent_rest],
                   ['no restrictions', 'early end to restrictions', 'intermittent restrictions']):
    data, actions = run_simulation(p)
    print(p.__name__)
    df = make_SIR_graph(data)
    actions_colors = list(map(color_map.get,  actions))
    make_beds_graph(df, actions_colors, 7, title, color_map)

100%|██████████| 280/280 [00:41<00:00,  6.79it/s]


unrestricted


100%|██████████| 280/280 [00:32<00:00,  8.66it/s]


unrestricted_hard_unrestricted


100%|██████████| 280/280 [00:28<00:00,  9.93it/s]


intermitent_rest
